In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import seaborn as sns
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
import torchvision
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer


In [4]:
train_dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/House Prices - Advanced Regression Techniques/train.csv')
test_dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/House Prices - Advanced Regression Techniques/test.csv')
# # Copying the original dataset
# dataset = home_price_dataset.copy()
# dataset.head()
train_dataset.head(4)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000


In [5]:
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [6]:
my_numeric_train_dataset = train_dataset.select_dtypes(exclude ='object') # obtains the numerical data type
my_numeric_train_dataset

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,61,0,0,0,0,0,2,2008,208500
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,298,0,0,0,0,0,0,5,2007,181500
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,42,0,0,0,0,0,9,2008,223500
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,35,272,0,0,0,0,2,2006,140000
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,192,84,0,0,0,0,0,12,2008,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,62.0,7917,6,5,1999,2000,0.0,0,...,0,40,0,0,0,0,0,8,2007,175000
1456,1457,20,85.0,13175,6,6,1978,1988,119.0,790,...,349,0,0,0,0,0,0,2,2010,210000
1457,1458,70,66.0,9042,7,9,1941,2006,0.0,275,...,0,60,0,0,0,0,2500,5,2010,266500
1458,1459,20,68.0,9717,5,6,1950,1996,0.0,49,...,366,0,112,0,0,0,0,4,2010,142125


In [7]:
my_numeric_test_dataset = test_dataset.select_dtypes(exclude ='object') # obtains the numerical data type
my_numeric_test_dataset

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,1461,20,80.0,11622,5,6,1961,1961,0.0,468.0,...,730.0,140,0,0,0,120,0,0,6,2010
1,1462,20,81.0,14267,6,6,1958,1958,108.0,923.0,...,312.0,393,36,0,0,0,0,12500,6,2010
2,1463,60,74.0,13830,5,5,1997,1998,0.0,791.0,...,482.0,212,34,0,0,0,0,0,3,2010
3,1464,60,78.0,9978,6,6,1998,1998,20.0,602.0,...,470.0,360,36,0,0,0,0,0,6,2010
4,1465,120,43.0,5005,8,5,1992,1992,0.0,263.0,...,506.0,0,82,0,0,144,0,0,1,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,21.0,1936,4,7,1970,1970,0.0,0.0,...,0.0,0,0,0,0,0,0,0,6,2006
1455,2916,160,21.0,1894,4,5,1970,1970,0.0,252.0,...,286.0,0,24,0,0,0,0,0,4,2006
1456,2917,20,160.0,20000,5,7,1960,1996,0.0,1224.0,...,576.0,474,0,0,0,0,0,0,9,2006
1457,2918,85,62.0,10441,5,5,1992,1992,0.0,337.0,...,0.0,80,32,0,0,0,0,700,7,2006


In [8]:
my_object_train_dataset = train_dataset.select_dtypes(include = 'object') # obtains all the datatypes that are objects
my_object_train_dataset

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1,RL,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
2,RL,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
3,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,...,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Abnorml
4,RL,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1456,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,...,Attchd,Unf,TA,TA,Y,NaN,MnPrv,NaN,WD,Normal
1457,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,...,Attchd,RFn,TA,TA,Y,NaN,GdPrv,Shed,WD,Normal
1458,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,...,Attchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Normal


In [9]:
my_object_test_dataset = test_dataset.select_dtypes(include = 'object') # obtains all the datatypes that are objects
my_object_test_dataset

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,RH,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,...,Attchd,Unf,TA,TA,Y,NaN,MnPrv,NaN,WD,Normal
1,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,...,Attchd,Unf,TA,TA,Y,NaN,NaN,Gar2,WD,Normal
2,RL,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,...,Attchd,Fin,TA,TA,Y,NaN,MnPrv,NaN,WD,Normal
3,RL,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,...,Attchd,Fin,TA,TA,Y,NaN,NaN,NaN,WD,Normal
4,RL,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,RM,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,...,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,WD,Normal
1455,RM,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,...,CarPort,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Abnorml
1456,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,...,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Abnorml
1457,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,...,NaN,NaN,NaN,NaN,Y,NaN,MnPrv,Shed,WD,Normal


In [10]:
#Adding YearBuilt,YrSold, YearRemodAdd, and GarageYrBlt to categorical dataframe
my_object_train_dataset[['YearBuilt', 'YrSold','YearRemodAdd','GarageYrBlt']] = my_numeric_train_dataset[['YearBuilt', 'YrSold','YearRemodAdd','GarageYrBlt']]
# Remove YearBuilt from numerical
my_numeric_train_dataset.drop(['Id','YearBuilt', 'YrSold','YearRemodAdd','GarageYrBlt'], axis=1, inplace=True)

In [11]:
#Adding YearBuilt,YrSold, YearRemodAdd and GarageYrBlt to categorical dataframe
my_object_test_dataset[['YearBuilt', 'YrSold','YearRemodAdd','GarageYrBlt']] = my_numeric_test_dataset[['YearBuilt', 'YrSold','YearRemodAdd','GarageYrBlt']]
# Remove YearBuilt from numerical
my_numeric_test_dataset.drop(['Id','YearBuilt', 'YrSold','YearRemodAdd','GarageYrBlt'], axis=1, inplace=True)

In [12]:
# Check for missing values
missing_values = my_numeric_test_dataset.isnull().sum()

# Fill missing values with the mean
num_data_filled_test = my_numeric_test_dataset.fillna(my_numeric_test_dataset.mean())

# Check if any missing values are left
missing_values_after_fill_test = num_data_filled_test.isnull().sum()

# Display the number of missing values before and after filling
print("Missing Values Before Filling:")
print(missing_values)
print("\nMissing Values After Filling:")
print(missing_values_after_fill_test)

Missing Values Before Filling:
MSSubClass         0
LotFrontage      227
LotArea            0
OverallQual        0
OverallCond        0
MasVnrArea        15
BsmtFinSF1         1
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
1stFlrSF           0
2ndFlrSF           0
LowQualFinSF       0
GrLivArea          0
BsmtFullBath       2
BsmtHalfBath       2
FullBath           0
HalfBath           0
BedroomAbvGr       0
KitchenAbvGr       0
TotRmsAbvGrd       0
Fireplaces         0
GarageCars         1
GarageArea         1
WoodDeckSF         0
OpenPorchSF        0
EnclosedPorch      0
3SsnPorch          0
ScreenPorch        0
PoolArea           0
MiscVal            0
MoSold             0
dtype: int64

Missing Values After Filling:
MSSubClass       0
LotFrontage      0
LotArea          0
OverallQual      0
OverallCond      0
MasVnrArea       0
BsmtFinSF1       0
BsmtFinSF2       0
BsmtUnfSF        0
TotalBsmtSF      0
1stFlrSF         0
2ndFlrSF         0
LowQualFinSF     0
GrLivA

In [13]:
# Check for missing values
missing_values = my_numeric_train_dataset.isnull().sum()

# Fill missing values with the mean
num_data_filled_train = my_numeric_train_dataset.fillna(my_numeric_train_dataset.mean())

# Check if any missing values are left
missing_values_after_fill_train = num_data_filled_train.isnull().sum()

# Display the number of missing values before and after filling
print("Missing Values Before Filling:")
print(missing_values)
print("\nMissing Values After Filling:")
print(missing_values_after_fill_train)

Missing Values Before Filling:
MSSubClass         0
LotFrontage      259
LotArea            0
OverallQual        0
OverallCond        0
MasVnrArea         8
BsmtFinSF1         0
BsmtFinSF2         0
BsmtUnfSF          0
TotalBsmtSF        0
1stFlrSF           0
2ndFlrSF           0
LowQualFinSF       0
GrLivArea          0
BsmtFullBath       0
BsmtHalfBath       0
FullBath           0
HalfBath           0
BedroomAbvGr       0
KitchenAbvGr       0
TotRmsAbvGrd       0
Fireplaces         0
GarageCars         0
GarageArea         0
WoodDeckSF         0
OpenPorchSF        0
EnclosedPorch      0
3SsnPorch          0
ScreenPorch        0
PoolArea           0
MiscVal            0
MoSold             0
SalePrice          0
dtype: int64

Missing Values After Filling:
MSSubClass       0
LotFrontage      0
LotArea          0
OverallQual      0
OverallCond      0
MasVnrArea       0
BsmtFinSF1       0
BsmtFinSF2       0
BsmtUnfSF        0
TotalBsmtSF      0
1stFlrSF         0
2ndFlrSF         0
LowQ

In [14]:
# # Separate features and target variable
# X = my_numeric_train_dataset.drop(columns=['SalePrice'])
# y = my_numeric_train_dataset['SalePrice']

In [15]:
X = num_data_filled_train.drop(columns=['SalePrice'])
y = np.log(num_data_filled_train[['SalePrice']].values).reshape(-1,1)


In [16]:
y

array([[12.24769432],
       [12.10901093],
       [12.31716669],
       ...,
       [12.49312952],
       [11.86446223],
       [11.90158345]])

In [17]:
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.2, random_state=42)

In [18]:
#Instantiate scaler
mm_scaler = MinMaxScaler()
ss_scaler = StandardScaler()

In [19]:
# Fit and Transform train dataset; Transform on X_test & X_val
X_train = mm_scaler.fit_transform(X_train)
# X_test = mm_scaler.transform(X_test)
X_val = mm_scaler.transform(X_val)

In [20]:
X_val

array([[0.        , 0.16780822, 0.03325154, ..., 0.        , 0.        ,
        0.09090909],
       [0.23529412, 0.26369863, 0.05120942, ..., 0.        , 0.        ,
        0.27272727],
       [0.05882353, 0.11986301, 0.03580359, ..., 0.        , 0.        ,
        0.18181818],
       ...,
       [0.23529412, 0.14383562, 0.03224661, ..., 0.        , 0.        ,
        0.81818182],
       [0.29411765, 0.13356164, 0.03638318, ..., 0.        , 0.        ,
        0.81818182],
       [0.        , 0.16780822, 0.03187735, ..., 0.        , 0.        ,
        0.54545455]])

In [21]:
X_train_tensor = torch.FloatTensor(X_train)
X_val_tensor = torch.FloatTensor(X_val)
y_train_tensor = torch.FloatTensor(y_train)
y_val_tensor = torch.FloatTensor(y_val)

In [22]:
X_train_tensor.shape

torch.Size([1168, 32])

In [23]:
train_da = TensorDataset(X_train_tensor, y_train_tensor)
val_da = TensorDataset(X_val_tensor, y_val_tensor)

In [24]:

batch_size = 10

train_loader = DataLoader(train_da, batch_size, shuffle=True)
val_loader = DataLoader(val_da, batch_size)

In [25]:
val_loader

In [26]:
input_size = X_train_tensor.shape[1]
output_size = 1

model = nn.Linear(input_size, output_size)

In [27]:
model.bias

Parameter containing:
tensor([-0.1408], requires_grad=True)

In [28]:
model.weight

Parameter containing:
tensor([[ 0.0973, -0.1636,  0.0235,  0.1468, -0.0256,  0.0781,  0.1626,  0.1500,
         -0.0757,  0.1003,  0.1661, -0.0748, -0.0228, -0.0255, -0.1593,  0.1323,
         -0.1517,  0.0069,  0.0968, -0.1268, -0.1326, -0.1593, -0.1400,  0.1765,
          0.1612, -0.0622, -0.1548, -0.0827,  0.1684,  0.1722, -0.1518,  0.1767]],
       requires_grad=True)

In [29]:
class RegModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(input_size,output_size)

  def forward(self, xb):
    out  = self.linear(xb)
    return out

  def training_step(self,batch):
    inputs, targets = batch
    outs = self(inputs)
    loss = F.l1_loss(input = outs, target = targets, size_average = None, reduce = None, reduction = 'mean')
    return loss

  def validation_step(self,batch):
    inputs, targets = batch
    outs = self(inputs)
    loss = F.l1_loss(input = outs, target = targets, size_average = None, reduce = None, reduction = 'mean')
    return {'val_loss': loss.detach()}

  def validation_epoch_end(self, outputs):
    batch_losses = [x['val_loss'] for x in outputs]
    epoch_loss = torch.stack(batch_losses).mean()
    return {'val_loss': epoch_loss.item()}

  def epoch_end(self, epoch, result, num_epochs):
    if (epoch+1) % 100 == 1 or epoch == num_epochs-1:
      print("Epoch [{}], val_loss: {:.2f}".format(epoch+1, result['val_loss']))

model = RegModel()

In [30]:
def evaluate(model, val_loader):
  outputs = [model.validation_step(batch) for batch in val_loader]
  return model.validation_epoch_end(outputs)

In [31]:
#function independent
def fit(epochs, model, train_loader, val_loader, lr, opt_func=torch.optim.SGD):
    optimizer = opt_func(model.parameters(), lr)
    history = []

    for epoch in range(epochs):
        # for training phase
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward() # compute gradients
            optimizer.step()
            optimizer.zero_grad()

        # for validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)

    return history

In [32]:
epochs = 1000
lr = 0.001

history = fit(epochs, model, train_loader, val_loader, lr)

print(history)

Epoch [1], val_loss: 11.84
Epoch [101], val_loss: 0.75
Epoch [201], val_loss: 0.46
Epoch [301], val_loss: 0.34
Epoch [401], val_loss: 0.29
Epoch [501], val_loss: 0.25
Epoch [601], val_loss: 0.22
Epoch [701], val_loss: 0.19
Epoch [801], val_loss: 0.17
Epoch [901], val_loss: 0.15
Epoch [1000], val_loss: 0.14
[{'val_loss': 11.8351411819458}, {'val_loss': 11.458256721496582}, {'val_loss': 11.081342697143555}, {'val_loss': 10.704462051391602}, {'val_loss': 10.327478408813477}, {'val_loss': 9.950600624084473}, {'val_loss': 9.573663711547852}, {'val_loss': 9.196768760681152}, {'val_loss': 8.819878578186035}, {'val_loss': 8.443020820617676}, {'val_loss': 8.066120147705078}, {'val_loss': 7.689181804656982}, {'val_loss': 7.312257766723633}, {'val_loss': 6.935368061065674}, {'val_loss': 6.558449745178223}, {'val_loss': 6.181499004364014}, {'val_loss': 5.804589748382568}, {'val_loss': 5.427690505981445}, {'val_loss': 5.050763130187988}, {'val_loss': 4.673876762390137}, {'val_loss': 4.2969684600830

In [33]:
result = evaluate(model, val_loader)
print(result)

{'val_loss': 0.14378248155117035}


In [34]:
def make_predictions(input, target, model):
    inputs = input.unsqueeze(0)
    predictions = model(inputs)
    prediction = predictions[0].detach()
    print("Target:", np.exp(target.item()))
    print("Prediction:", np.exp(prediction.item()))

In [40]:
input, target = val_da[126]
make_predictions(input, target, model)

Target: 235999.93288487458
Prediction: 212682.80240642882
